In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
import numpy as np
import netket as nk
import jax

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/user/miniconda3/envs/rbm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import sys
sys.path.insert(0, "../")
from spinlattice.lattice import get_lattice_data, LatticeData



In [23]:
lt_info = get_lattice_data("dimer-hexagonal-lattice", "dimer-hexagonal", [2, 2,], "periodic")

Bond Types and Bonds:
Loop Types and Loops:


In [92]:
from abc import ABC, abstractmethod
from netket.operator.spin import sigmax,sigmaz, identity

class DimerModel(ABC):
    def __init__(self, lattice_data: LatticeData):
        self.lattice_data = lattice_data
        self.n_sites = lattice_data.n_sites()
        self.hi = nk.hilbert.Spin(s=1 / 2, N=self.n_sites)

    @abstractmethod
    def projection(self):
        pass


    # @abstractmethod
    def hamiltonian(self, V : float, h : float):
        """
        V : float
            The strength of the dimer-dimer interaction
        h : float
            The strength of the dimer-flip
        """
        pass

    @abstractmethod
    def _dimer_potential_local(self, V : float):
        """
        Return the local operator for the dimer-dimer interaction.

        The definition depends on the lattice
        """
        pass

    def dimer_potential(self, V : float):
        return sum([self._dimer_potential_local(b) for b in range(self.lattice_data.n_bonds())])

    def dimer_flip(self, h : float):
        """
        Return the system operator for the dimer-flip term
        """
        return sum([self._dimer_flip_local(b) for b in range(self.lattice_data.n_bonds())])


    def _dimer_flip_local(self, b : int):
        """
        Flips the dimer at bond b
        """
        bond = self.lattice_data.bonds[b]
        return sigmax(self.hi, bond[0]) * sigmax(self.hi, bond[1]) 

class DimerHexagonal(DimerModel):
    def __init__(self, lattice_data: LatticeData):
        super().__init__(lattice_data)
    
    def projection(self):
        loops = self.lattice_data.loops
        bonds = self.lattice_data.bonds
        bond_colors = self.lattice_data.bond_types

        _projection_bonds = []
        _projection_bond_coords = []

        self.projections = []
        for loop in loops:
            bi = np.where(np.isin(bonds, loop).all(axis=1))[0] # bond indices
            assert len(bi) == 6
            local_projection = sum([((-1) ** c) * sigmaz(self.hi, e[0]) * sigmaz(self.hi, e[1]) for e, c in zip(bonds[bi], bond_colors[bi])])
            local_projection -= 4 * identity(self.hi)
            self.projections.append(local_projection)
        
        return sum(self.projections)
    
    def _dimer_potential_local(self, b : int):
        """
        Return the local operator for the dimer-dimer interaction.
        """
        bonds = self.lattice_data.bonds
        bond = bonds[b]
        coords = self.lattice_data.coordinates  

        b1 = bond[0]
        b2 = bond[1]

        conn_b1 = np.where(np.isin(bonds, b1).any(axis=1))[0]
        conn_b1 = conn_b1[conn_b1 != b]
        sites_conn_b1 = np.unique(bonds[conn_b1].reshape(-1))
        sites_conn_b1 = sites_conn_b1[sites_conn_b1 != b1]

        vec_b1 = coords[sites_conn_b1] - coords[b1]

        sym_b1 = coords[b2] - vec_b1

        # search for the coordinates
        sym_b1 = np.where(np.isin(coords, sym_b1).all(axis=1))[0]

        conn_b2 = np.where(np.isin(bonds, b2).any(axis=1))[0]
        conn_b2 = conn_b2[conn_b2 != b]
        sites_conn_b2 = np.unique(bonds[conn_b2].reshape(-1))
        sites_conn_b2 = sites_conn_b2[sites_conn_b2 != b2]

        # vec_b2 = coords[sites_conn_b2] - coords[b2]




        # b2 = bond[1]
        # conn_b2 = np.where(np.isin(bonds, b2).any(axis=1))[0]
        # conn_b2 = conn_b2[conn_b2 != b]
        # print(conn_b2)


In [93]:
dimer = DimerHexagonal(lt_info)
dimer_projection = dimer.projection()
dimer._dimer_potential_local(0)

[[1.83333333 0.28867513]
 [1.83333333 0.8660254 ]]


[ 0  7 15]


In [59]:
dimer_projection.get_conn(np.ones(dimer.n_sites))

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([0.]))

In [ ]:
b = np.zeros_like(bonds, dtype=bool)
np.where(np.isin(bonds, loops[0]).all(axis=1))[0]


In [16]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"


In [17]:
N = lt_info.n_sites()
hi = nk.hilbert.Spin(s=1 / 2, N=N)

In [8]:
from netket.operator.spin import sigmax,sigmaz, identity


Gamma=[1, -1]
H = sum([Gamma[c]*sigmax(hi,e[0]) * sigmax(hi, e[1]) for e, c in zip(lt_info.bonds, lt_info.bond_types)])
I = identity(hi)


In [10]:
I._setup()
I.get_conn(np.ones(N))

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([1.]))

In [11]:
H.get_conn(np.ones(N))



(array([[-1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
         -1.,  1.,  1.],
        [-